# 5.회귀분석

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import bartlett, levene, shapiro
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from sklearn.datasets import load_boston

sklearn 패키지의 datasets 패키지는 회귀분석용 예제 제공 <br>
load_boston()은 보스턴 주택 가격 데이터 예측을 위한 데이터 셋 <br> 보스턴의 506개 타운 13개 독립변수 값으로 해당 타운의 주택가격 중앙값을 예측. 

> 독립변수 <br>
CRIM: 범죄율 <br>
INDUS: 비소매상업지역 면적 비율 <br>
NOX: 일산화질소 농도 <br>
RM: 주택당 방 수 <br>
LSTAT: 인구 중 하위 계층 비율 <br>
B: 인구 중 흑인 비율 <br>
PTRATIO: 학생/교사 비율 <br>
ZN: 25,000 평방피트를 초과 거주지역 비율 <br>
CHAS: 찰스강의 경계에 위치한 경우는 1, 아니면 0 <br>
AGE: 1940년 이전에 건축된 주택의 비율 <br>
RAD: 방사형 고속도로까지의 거리 <br>
DIS: 직업센터의 거리 <br>
TAX: 재산세율 <br>

>종속변수 <br>
보스턴 506개 타운의 1978년 주택 가격 중앙값 (단위 1,000 달러)

## 5.1 EDA

데이터 분석 전 그래프나 통계적인 방법으로 자료를 직관적으로 바라보는 과정 <br>
주로 시각화를 통한 방법을 사용 <br>
<br>

최소한 아래의 내용 확인 <br>
- 데이터의 이상치, 결측치 등을 확인
- 데이터의 분포가 어떠한지 확인
- 데이터 간 패턴 등을 파악
- 다중 공선성 파악

In [6]:
data = load_boston()

In [8]:
data.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])

In [ ]:
print(data.get('target'))

In [20]:
pd.DataFrame(data.get('data'), columns=data.get('feature_names'))

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [21]:
a = pd.DataFrame(data.get('data'), columns=data.get('feature_names'))

In [23]:
a['target'] = data.get('target')
a

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [24]:
pd.concat([
    pd.DataFrame(data.get('data'), columns=data.get('feature_names')),
    pd.DataFrame(data.get('target'), columns=['target'])
])

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,NaN
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,NaN
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,NaN
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,NaN
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.4
502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.6
503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.9
504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0


In [25]:
pd.concat([
    pd.DataFrame(data.get('data'), columns=data.get('feature_names')),
    pd.DataFrame(data.get('target'), columns=['target'])
], axis=1)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [26]:
boston_data = pd.concat([
    pd.DataFrame(data.get('data'), columns=data.get('feature_names')),
    pd.DataFrame(data.get('target'), columns=['target'])
], axis=1)  # a['target'] = data.get('target')

In [27]:
boston_data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


## 5.2 Modeling

모델을 구축하는 과정으로 EDA를 통해 확인한 내용을 반영하여 모델 구축 <br>

statsmodel에서 두 가지 방법으로 사용 가능
```python
model = sm.OLS(y, x).fit()
```

```python
model = sm.OLS.from_formula("y ~ x", data).fit()
```
<br>

위의 from_formula는 다양한 활용 가능 <br>
- scale 
```python
model = sm.OLS.from_formula("y ~ scale(x)", data).fit()
```
- $x^2$
```python
model = sm.OLS.from_formula("y ~ I(x ** 2)", data).fit()
```
- 범주화
```python
model = sm.OLS.from_formula("y ~ C(x)", data).fit()
```
<br>



모델 결과는 아래의 내용으로 해석
- R-squared
설명하고자 하는 y를 주어진 독립변수로 얼마 정도 설명할 수 있는지 나타내는 지표 <br>
범위는 0에서 1사이의 값으로 0이면 모델의 설명력이 전혀 없는 상태이고, 1에 가까울수록 모델이 데이터를 잘 설명
- Adj. R-squared
R-squared와 내용은 같으나 R-square는 독립변수가 유의한지 아닌지와 간계 없이 추가할수록 값이 증가한다는 단점이 있음 <br>
이러한 단점을 보완하기 위해 독립변수의 숫자로 R-square 값을 보정
- F-statistic
모형의 적합도를 나타내는 통계량
- Prob (F-statistic) 
위에서 구한 F-statistic의 확률값 <br>
유의수준과 비교하여 그보다 작으면 모형이 유효
<br>

- coef
데이터로 추정한 각 독립변수의 계수
- t
추정 계수의 t 통계량
- P>|t|  
p-value라고도 하며 위의 통계량을 통해 얻은 값 <br>
유의수준과 비교하여 유의수준보다 작으면 해당 추정 계수가 유의
- [0.025      0.975]
해당 계수의 95% 신뢰구간에 해당하는 상, 하한 값 <br>
만일 이 범위에 0을 포함하면 해당 계수는 유의하지 않다고 판단
<br>

- Omnibus <br>
정규성 테스트

- Durbin-Watson <br>
잔차의 독립성을 나타내는 통계량 <br>
0: 양의 자기 상관 <br>
2: 독립 <br>
4: 음의 자기 상관 <br>

- Jarque-Bera (JB) <br>
샘플의 Skewness와 Kurtosis를 사용하여 정규분포가 맞는지 아닌지 검증

- Cond. No. <br>
가장 큰 고유치와 가장 작은 고유치의 비율 <br>
모델 결과에서 큰 조건수를 갖는 경우는 다음과 같음 <br>
1) 변수들의 단위 차이로 인해 숫자의 스케일이 크게 달라지는 경우 <br>
\-> scaling으로 해결 <br>
2) 다중 공선성 변수들이 있는 경우 <br>
\-> 변수 선택이나 PCA를 사용한 차원 축소 등으로 해결

## 5.3 모형검증
만든 모델이 통계적으로 적절한 모델인지 확인하는 작업 <br>
아래의 4가지를 확인하는 절차를 가짐 <br>

### 선형성

독립 변수와 종속 변수 간에는 선형적 관계를 지니고 있어야 함 <br>

<br>

확인방법 <br>
- Plot <br>

<br>

만일 선형성을 만족하지 않을 경우 아래의 방법을 사용할 수 있음 <br>
- 새로운 변수 추가 <br>
- 변수 변환 <br> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ex) log, exp, sqrt, etc. <br>
- 선형성을 만족하지 않는 변수제거 <br>


### 정규성

회귀모델의 잔차가 정규분포를 따라야 한다는 가정

<br>

확인방법 <br>
- shapiro-wilk test
- QQ plot (scipy.stats.probplot)

<br>

정규성을 만족하지 않을 시 아래의 방법 사용 가능 <br>
- 변수 변환 <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ex) box-cox transform
- Maximum Likelihood Estimation

### 독립성

독립변수간 상관관계가 없어야 한다는 가정 <br>
<br>

확인 방법
- Durbin Watson Test

<br>

- 회귀분석 결과의 durbin watson 통계량으로 확인
- 0~4 사이의 값을 가짐
- 2는 독립
- 0 ~ d 양의 상관관계
- 2+d ~ 4 음의 상관관계

### 등분산성

회귀모델의 잔차가 고른 분산을 가지고 있어야 한다는 가정

<br>

확인 방법 <br>
- plot
- levene과 bartlett (파이썬에서 공식적인 방법은 아님)
  - levene: 정규분포 가정 <br>
  - bartlett: 모든 분포에 사용 <br>
<br>

등분산을 만족하지 않을 시 아래의 방법 사용 가능
- WLS regression
- GLS regression

### 5.4 Toy Project

In [ ]:
!pip install finance-datareader

In [4]:
import pandas as pd
import FinanceDataReader as fdr
import statsmodels.api as sm
import seaborn as sns


In [5]:
NAVER_STOCK_CODE = '035420'

In [6]:
fdr.DataReader(NAVER_STOCK_CODE)

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2002-10-29,2312,2312,2312,2312,20549,NaN
2002-10-30,2587,2587,2558,2588,853638,0.119377
2002-10-31,2626,2689,2398,2459,1320880,-0.049845
2002-11-01,2520,2599,2215,2305,750761,-0.062627
2002-11-04,2285,2300,2135,2191,692165,-0.049458
...,...,...,...,...,...,...
2023-01-19,194000,196500,192500,196000,475600,0.002558
2023-01-20,195500,201000,195000,196000,787881,0.000000
2023-01-25,200500,203500,199600,202000,988822,0.030612


In [7]:
fdr.DataReader(NAVER_STOCK_CODE).loc['2013':'2022']

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2013-01-02,72354,72982,71256,71413,154535,0.002203
2013-01-03,71881,72823,68743,69372,356250,-0.028580
2013-01-04,69529,71099,69215,71099,234701,0.024895
2013-01-07,72039,75492,71882,75336,404906,0.059593
2013-01-08,75336,75807,74081,74395,198565,-0.012491
...,...,...,...,...,...,...
2022-12-23,180500,182000,178000,178000,613209,-0.035230
2022-12-26,178500,180500,177000,178500,350843,0.002809
2022-12-27,179500,182500,179000,181000,492232,0.014006


In [17]:
financial_statement = pd.read_csv(r'C:\Users\user\Documents\bigdata\My Coding\2022_11_19_bigdata\bigdata2\data\financial_statement_naver.csv').iloc[:,1:]

In [21]:
financial_statement

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,...,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord,currency,thstrm_add_amount,frmtrm_add_amount
0,20160330003610,11011,2015,266961,35420,CFS,연결재무제표,BS,재무상태표,유동자산,...,제 16 기,2014.12.31 현재,"1,945,783,670,000",제 15 기,2013.12.31 현재,"1,336,088,790,000",1,KRW,NaN,NaN
1,20160330003610,11011,2015,266961,35420,CFS,연결재무제표,BS,재무상태표,비유동자산,...,제 16 기,2014.12.31 현재,"1,448,674,224,000",제 15 기,2013.12.31 현재,"1,361,658,048,000",3,KRW,NaN,NaN
2,20160330003610,11011,2015,266961,35420,CFS,연결재무제표,BS,재무상태표,자산총계,...,제 16 기,2014.12.31 현재,"3,394,457,894,000",제 15 기,2013.12.31 현재,"2,697,746,838,000",5,KRW,NaN,NaN
3,20160330003610,11011,2015,266961,35420,CFS,연결재무제표,BS,재무상태표,유동부채,...,제 16 기,2014.12.31 현재,"1,080,709,107,000",제 15 기,2013.12.31 현재,"832,373,207,000",7,KRW,NaN,NaN
4,20160330003610,11011,2015,266961,35420,CFS,연결재무제표,BS,재무상태표,비유동부채,...,제 16 기,2014.12.31 현재,"516,330,572,000",제 15 기,2013.12.31 현재,"390,064,617,000",9,KRW,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,20221114002234,11014,2022,266961,35420,OFS,재무제표,BS,재무상태표,자본총계,...,제 23 기말,2021.12.31 현재,"9,332,458,651,752",NaN,NaN,NaN,22,KRW,NaN,NaN
710,20221114002234,11014,2022,266961,35420,OFS,재무제표,IS,손익계산서,매출액,...,제 23 기3분기,2021.01.01 ~ 2021.09.30,"1,264,534,001,915",NaN,NaN,NaN,24,KRW,"4,085,720,083,959","3,645,588,257,320"
711,20221114002234,11014,2022,266961,35420,OFS,재무제표,IS,손익계산서,영업이익,...,제 23 기3분기,2021.01.01 ~ 2021.09.30,"398,360,266,811",NaN,NaN,NaN,26,KRW,"1,178,967,600,491","1,149,660,200,193"
712,20221114002234,11014,2022,266961,35420,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,...,제 23 기3분기,2021.01.01 ~ 2021.09.30,"591,530,960,379",NaN,NaN,NaN,28,KRW,"1,648,286,883,869","1,487,855,129,325"


In [ ]:
financial_statement.query('bsns_year == 2015')

In [28]:
financial_statement_2013 = (
    financial_statement
    .query('bsns_year == 2015')
    .filter(items=['rcept_no','bsns_year','corp_code','stock_code','sj_nm','account_nm','thstrm_amount'])
)

financial_statement_2013.bsns_year = 2013
